# SPECFEM Users Workshop -- Day 3 (Oct. 7, 2022)

## Part 3b: Seismic Imaging Exercise  

- In this notebook we perform a multi-event, multi-station 2D inversion
- We will perform all book keeping tasks manually, from running forward simulations to running adjoint simulations  
- **Objective**: Heavy-handidly illustrate the laborious book keeping required as inversion problems scale, prompting the need for automated tools    
- These instructions should be run from inside a Docker container, using Jupyter Lab (see instructions [here](https://github.com/adjtomo/adjdocs/blob/main/readmes/docker_image_install.md)).  
-----------

**Relevant Links:** 
- Today's Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_3a_imaging_adv.ipynb
- Completed Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_3a_imaging_adv.ipynb
- Day 0 Notebook (Container Testing): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_0_container_testing.ipynb
- Day 1A Notebook (Intro SPECFEM): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1a_intro_specfem2d.ipynb
- Day 1B Notebook (Fwd. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_1b_forward_simulations.ipynb
- Day 2A Notebook (Adj. Simulations): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_2a_kernels.ipynb
- Day 3A Notebook (Simple Imaging): https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/completed_notebooks/day_3a_imaging_simple.ipynb


**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.

----------------

# A Multi-event, Multi-station Inversion

- Real seismic inversions use multiple events recorded at multiple stations to increase coverage and reduce nonuniqueness
- Here we attempt a *multi-event* ($N=10$), multi-station ($S=10$) model update with the skills we have picked up in previous notebooks
- The inversion workflow is as follows: 
    0) **data**: run $N$ forward simulations to generate *data* using target model ($m_{true}$) for $N$ events and $S$ stations
    1) **synthetics**: run $N$ forward simulations to generate synthetics through starting model ($m_{init}$) for $N$ events and $S$ stations
    2) **misfit**: quantify data-synthetic misfit ($\chi_{init}$) and generate adjoint sources for each source-receiver pair ($N\times S$ waveforms)
    3) **kernels**: generate misfit kernels with $N$ adjoint simulation
    4) **update**: scale gradient and update initial model to get trial model: $m_{try}$
    5) **line search**: run $N$ forward simulations using trial model $m_{try}$ to get $N \times S$ new synthetic waveforms
    6) **line search**: calculate misfit ($\chi_{try}$) and determine if misfit is reduced ($\chi_{try} < \chi_{init}$?)
    7) **line search**: if misfit is not reduced ($\chi_{try} >=  \chi_{init}$), repeat steps 5-7 with newly scaled gradient
    8) **line search**: if misfit is reduced, accept trial model as newly updated model ($m_{try} \rightarrow m_{01}$)
    9) **finalize**: set newly updated model as initial model ($m_{01} \rightarrow m_{init}$) and repeat steps 1-9

In [ ]:
# Python packages we might need for today's notebook. You are welcome to import more as you see fit
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from IPython.display import Image
from scipy.integrate import simps
from seisflows.tools.specfem import Model, read_fortran_binary, write_fortran_binary

## 1) Set Up a SPECFEM2D Working Directory

- We want to set up a clean working directory to run SPECFEM2D. This will help us preserve our cloned repository and reduce file clutter  
- During this example we will use: 10 sources, 10 receivers
- *Starting model*: homogeneous halfspace  
- *Target model*: perturbation checkerboard  
- **Objective**: Generate a misfit kernel and manually update a 2D model, quantify misfit 
- **NOTE:** We will work in `/home/scoped/work/day_3/inversion_example`. The following cells assume relative paths, so you must evaluate the '%cd' command below.

In [ ]:
# Make sure we're in an empty working directory
! rm -rf /home/scoped/work/day_3/inversion_example
! mkdir -p /home/scoped/work/day_3/inversion_example
%cd /home/scoped/work/day_3/inversion_example

# Setup SPECFEM2D working directory
! ln -s /home/scoped/specfem2d/bin .
! cp -r /home/scoped/specfem2d/EXAMPLES/Tape2007/DATA .
! mkdir OUTPUT_FILES

### 1a) Visualize Experimental Setup

- We plot the 10 sources and 10 receivers on top of the checkerboard model for context  
- The initial model is a homogeneous halfspace, and is **not** plotted here

In [ ]:
def plot_sources_receivers(nevents=25):
    """
    Small code block to plot source-receiver geometry with text labels
    Assumes relative pathing
    """
    # Grab Station locations
    sta_x, sta_z = np.genfromtxt("DATA/STATIONS_checker", dtype=float, usecols=[2, 3]).T
    
    # Grab Event locations
    ev_x, ev_z = [], []
    for i in range(1, nevents+1):
        source_file = f"DATA/SOURCE_{i:0>3}"
        with open(source_file, "r") as f:
            lines = f.readlines()
        # Trying to break apart the following line
        # 'xs = 299367.72      # source location x in meters\n'
        xs = float(lines[2].split("=")[1].split("#")[0].strip())
        zs = float(lines[3].split("=")[1].split("#")[0].strip())

        ev_x.append(xs)
        ev_z.append(zs)
        
    # Plot SOURCES and STATIONS geometry
    for i, (x, z) in enumerate(zip(sta_x, sta_z)):
        plt.scatter(x, z, c="c", marker="v", s=60, edgecolor="k")
        # plt.text(x, z, f"R{i:0>2}", fontsize=12)
    for i, (x, z) in enumerate(zip(ev_x, ev_z)):
        plt.scatter(x, z, c="y", marker="*", edgecolor="k", s=150)
        plt.text(x, z, f"S{i+1:0>2}", fontsize=12, c="k")  # SOURCE numbering starts at 1
    
    # Finalize plot labels
    plt.xlabel("X [m]")
    plt.ylabel("Z [m]")
    plt.xlim([0, 480E3])
    plt.ylim([0, 480E3])
    plt.title("SOURCE-RECEIVER GEOMETRY")
    
    
# Load and plot checkerboard model
data = np.genfromtxt("DATA/model_velocity.dat_checker", dtype=float, usecols=[1,2,4,5])
chkbd_x, chkbd_z, chkbd_vp, chkbd_vs = data.T
plt.tricontourf(chkbd_x, chkbd_z, chkbd_vp, levels=125, cmap="seismic_r", alpha=0.1)

# Plot sources and receivers on top
plot_sources_receivers()
plt.show()

-----------
## 2) Target Model (checkerboard) Synthetics 

- We'll use the Target checkerboard model used to generate 'data'
- Let's first visualize it to help us contexualize the problem  

### 2a) Set up the Target Model
- We want to set up our forward simulations here  
- We'll be using the Checkerboard model to generate 'data'  

#### **TASKS TO COMPLETE**:
1) Set the correct `DATA/Par_file` for the checkerboard model (*remember* that the checkerboard model parameters are defined in `Par_file_Tape2007_132rec_checker`)
2) Tell SPECFEM to save our model in FORTRAN binary format (in the `Par_file`)  
3) Tell SPECFEM to save our DATABASE files in FORTRAN binary format (in the `Par_file`)  
4) Tell SPECFEM to use our existing STATIONS file, rather than it's own internal definition (in the `Par_file`)  
5) Set the number of time steps (NSTEP) to 5000  
6) Set the correct naming convention for our checkerboard model, which is currently named `model_velocity.dat_checker` (see Notebook 2A Section 3A)
7) Create a STATIONS file with 10 stations (easiest solution is to copy the first ten lines of `STATIONS_checker`)  

### 2b) Run the Forward Solver

- We want to run 10 simulations to generate 10 sets of target data  
- That means we need to run the forward solver **once** per event (i.e., 10 times)
- *Remember* that each time we run SPECFEM, it overwrites existing files in the OUTPUT_FILES/ directory

#### **TASK TO COMPLETE:**

- Run the mesher and solver for **ten** (10) different events  
- Use SOURCE_001 -> SOURCE_010 (or any ten sources of your choosing)  
- Only use **one** core when running the mesher and solver  
- After each solver run, make sure you **store** your synthetic seismograms somewhere they can't be overwritten


----------
## 3) Initial Model Synthetics

- Now let's generate synthetics through a homogeneous halfspace initial model  
- **IMPORTANT**: We need to save the forward wavefield (parameter `SAVE_FORWARD`) from the forward simulation of each $N$ events  
- *Reminder:* The saved forward wavefield is a snapshot of the forward wavefield at the last time step  

### 3a) Set up the Initial Model
- We want to set up our forward simulations here  
- We'll be using the homogeneous halfspace model to generate synthetics    

#### **TASKS TO COMPLETE:**
1) Set the correct `Par_file` for the homogeneous halfspace (*remember* homogeneous halfspace is defined by `Par_file_Tape2007_onerec`)
2) Tell SPECFEM to save our model in FORTRAN binary format (in the `Par_file`)  
3) Tell SPECFEM to save our DATABASE files in FORTRAN binary format (in the `Par_file`)  
4) Tell SPECFEM to use our existing STATIONS file, rather than it's own internal definition (in the `Par_file`)  
5) Tell SPECFEM to **save** the forward wavefield after our simulation

### 3b) Run the Forward Solver

- We want to run 10 simulations to generate 10 sets of synthetics
- That means we need to run the forward solver **once** per event (i.e., 10 times)
- *Remember* that each time we run SPECFEM, it overwrites existing files in the OUTPUT_FILES/ directory

#### **TASK TO COMPLETE:**

- Run the mesher and solver for **ten** (10) different events  
- Use the **same** events that you used in Section 2
- Only use **one** core when running the mesher and solver  
- After each solver run, make sure you **store** your synthetic seismograms somewhere they can't be overwritten
- Also make sure to **store** the forward wavefield which has been output by SPECFEM  

-------------
## 4) Quantify Misfit, Generate Adjoint Sources for Initial Model

- We want to quantify misfit for all $N \times S$ waveforms
- We also want to generate $N \times S$ adjoint sources for subsequent adjoint simulations
- We'll use a waveform difference misfit for simplicity
- Misfit: $ \chi = \frac{1}{2} \int [d(t)-s(t)]^2 dt~$
- Adjoint Source: $f^\dagger (t) = s(t) - d(t)$

### 4a) Quantify Misfit and Generate Adjoint Sources

#### **TASKS TO COMPLETE:**

1) For a given **event** and for each **station** load in 'data' and synthetics  
2) Calculate the misfit value: $ \chi = \frac{1}{2} \int [d(t)-s(t)]^2 dt~$, and store its value in a text file (e.g., *'source_001_misfit_initial_model.txt'*)  
3) Calculate the adjoint source: $f^\dagger (t) = s(t) - d(t)$
4) Save the adjoint source with the correct format (ASCII) and the correct filename  
5) Store adjoint sources per-event (i.e., SOURCE_001 should have it's own directory of adjoint sources)  
6) Optional: Plot data and synthetics alongside adjoint sources  

--------
## 5) Run Adjoint Simulations for Event Kernels

- Now we need to run the adjoint simulations to generate event kernels
- *Remember* that event kernels tell us what parts of the model are sensitive to the misfit function for **EACH** event
- **NOTE**: We need to place the adjoint sources in the correct location and also tell SPECFEM that we are running an adjoint simulation
- SPECFEM expects adjoint sources in the `SEM/` directory, and formatted exactly the same as its output synthetics

#### **TASKS TO COMPLETE:** 

1) For a given SOURCE that you used to run your forward simulations  
2) Make sure that the correct SOURCE is set in the *DATA/* directory  
3) Create a '*SEM/*' directory and make sure the correct adjoint sources are linked  
4) Make sure the correct forward wavefield (from *save_forward*) is accesible in the 'OUTPUT_FILES/' directory 
5) Tell SPECFEM that we want to run an adjoint simulation (*simulation_type*)
6) Run the adjoint solver with 1 core
7) Store the output kernel files (*\*_kernel.bin*) somewhere safe so they are not overwritten by your next simulation  
7) Repeat steps 1-6 for all other SOURCES used for your forward simulations  

----------
## 6) Sum Event Kernels 

- Now we have $N$=10 event kernels, each representing misfit sensitivity for each of the $N$ events
- We need to sum the *event kernels* into a *misfit kernel*, which represents misfit sensitivity for **all** $N$ events
- We can use the `xcombine_sem` executable to sum these kernels into the **gradient**

#### **TASK TO COMPLETE**:

- Combine the `alpha` and `beta` kernels for **all** ten events using the executable `xcombine_sem`
- Remember that the command line call of `xcombine_sem` is:

```bash
mpirun -n NPROC bin/xcombine_sem KERNEL_NAMES INPUT_FILE OUTPUT_DIR
```

Where:
- `KERNEL_NAMES`: the name of the kernels you want to smooth, corresponding to the filenames (e.g., proc000000_alpha_kernel.bin corresponds to 'alpha_kernel')    
- `INPUT_FILE`: a text file that lists the full paths to **all** your kernel files. You should have 10 lines per kernel name since we have 10 events  
- `OUTPUT_DIR`: a directory where SPECFEM can output the **gradient**    

----------
## 7) Smoothing/Regularizing Gradient

- We want to use regularization to smooth out the gradient to get a more conservative picture of model updates  
- We use the `xsmooth_sem` executable to smooth our kernels  
- We should choose smoothing length based on the shortest period of our data  
- Because we know the target model, we smooth conservatively to the length scale of the checkerboards ($\Gamma$=50km)  
- The SPECFEM smoothing code convolves the kernel with a Gaussian. We are allowed to choose the half-width of the Gaussian  
- The relationship between half-width and full-width is $\Gamma\approx2.355\sigma$, so we choose $\sigma$=20km  

 The usage of `xsmooth_sem` is given as
 ```bash
 USAGE:  mpirun -np NPROC bin/xsmooth_sem SIGMA_H SIGMA_V KERNEL_NAME INPUT_DIR OUPUT_DIR GPU_MODE
 ```
 We will need to choose values and directories to make this work
  - `SIGMA_H`: Horizontal smoothing length [m] representing the horizontal half-width of the Gaussian  
  - `SIGMA_Z`: Vertical smoothing length [m] representing the vertical half-width of the Gaussian  
  - `KERNEL_NAME`: The name of the kernel we want to smooth. Must match filename, so `proc000000_vs_kernel.bin` will correspond to kernel name `vs_kernel`  
  - `INPUT_DIR`: where SPECFEM can find the kernel files
  - `OUTPUT_DIR`: Where SPECFEM should output the SMOOTHED kernels 
  - `GPU_MODE`: Use GPU acceleration to speed up the smoothing operation (.true. or .false.)
  
#### **TASKS TO COMPLETE:**

1) Make sure all your model files (located in *DATA/\*.bin*) and gradient files (located where `xcombine_sem` outputted them) are in the same place 
2) Smooth your `alpha` and `beta` kernels by 20km in the horizontal and vertical directions  
3) Remember to keep track of your smoothed and raw gradient. We will proceed with our **smoothed** gradient  

-------------
## 8) Scaling the Gradient and Perturbing the Initial Model

- We need to **scale** the gradient before using it to perturb the initial model  
- We can use SeisFlows utility functions to read in the gradient files  

#### **TASKS TO COMPLETE:**
- Copy and adjust the PATHS and complete tasks in this code block before running it
- **Only** update seismic velocity Vp and Vs  (during your own research, other quantities may be updated)  


```python
# Copy and edit the code below
gradient_vp = read_fortran_binary(PATH_TO_ALPHA_SMOOTH)
gradient_vs = read_fortran_binary(PATH_TO_BETA_SMOOTH)

model_init_vp = read_fortran_binary(PATH_TO_MODEL_VP)
model_init_vs = read_fortran_binary(PATH_TO_MODEL_VS)

# TASKS TO COMPLETE HERE
# 1) SCALE the gradient so that it is at max 10% of the PEAK Vp or Vs value
# 2) SUBTRACT the gradient from its respective model in new parameters: 'model_01_vs', 'model_01_vp'
# 3) Match the formatting of the 'write' functions below, which will OVERWRITE our initial model

# Overwrites Vp and Vs for Model 01 with perturbed model parameters
write_fortran_binary(model_01_vp, "DATA/proc000000_vp.bin")
write_fortran_binary(model_01_vs, "DATA/proc000000_vs.bin")
```

-----------------
## 9) Run Forward Simulations with Updated Model

- Now that we have a perturbed model (Model 01) we can use it to run forward simulations
- We'll calculate misfit for each of the new synthetics 
- We can compare the overall misfit of the initial model to Model 01


#### **TASKS TO COMPLETE:**

1) Tell SPECFEM that we want to run forward simulations again  
2) Tell SPECFEM that we want to use a 'gll' model (`model`=gll)
3) Choose one of your 10 events that you used to run your original forward simulations    
4) Run mesher and solver
5) Quantify misfit for each set of synthetics, save misfit to a text file (e.g., *'source_001_misfit_updated_model.txt'*)  
6) Repeat Steps 3-5 for all events  

### 9a) Optional: Visualize Waveform Differences

#### **TASKS TO COMPLETE**
- Plot initial model synthetics against updated model synthetics for one or all synthetics, to check that waveforms have changed  

----------------
## 10) Compare Misfit between Initial and Updated models
- Now we have two files containing misfit, one for the initial model, and one for the updated model (M01)  
- Each line in the misfit file corresponds to the overall misfit for a single event  
- We can compare them by taking the average of all lines in the misfit file  


#### **TASKS TO COMPLETE:**
1) Gather all misfit text files for your initial model
2) For one given event, read in the text file and sum all the lines in the file (summing misfit from all stations)  
3) Divide this value by the number of stations (or number of lines)  
4) Repeat steps 1-3 for all events  
5) Sum all values gathered in (4) 
6) Divide number retrieved in line 5 by the number of events  
7) Repeat Steps 1-6 for updated model
8) Compare recovered misfit value between initial and updated model  